In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import math

In [3]:
# data structure container
class Value:
    def __init__(self, data, _children=(), _op='', label=""):
        self.data = data
        self.grad = 0.0
        self._backward = lambda: None
        self._prev = set(_children) # to store the intermidiate results of the bigger operation
        self._op = _op # to know which operation caused that intermidiate result
        self.label = label

    # representation container
    # if not defined then it will show memory location like <__main__.Value at 0x788a537fe810>
    def __repr__(self):
        return f"Value(data={self.data})"

    # arithmetic operation
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        output = Value(self.data + other.data, (self, other), '+')

        def _backward():
            self.grad += (1 * output.grad)
            other.grad += (1 * output.grad)
        output._backward = _backward
        return output

    def __sub__(self, other): # self - other
        return self + (-other)

    def __neg__(self): # -self
        return self * -1

    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        output = Value(self.data * other.data, (self, other), '*')

        def _backward():
            self.grad += (other.data * output.grad)
            other.grad += (self.data * output.grad)
        output._backward = _backward
        return output

    # fallback method to perform '3*a' kind of operation
    def __rmul__(self, other): # other * self
        return self*other

    def __pow__(self, other): 
        assert isinstance(other, (int, float)), "only supports int/floats"
        output = Value(self.data ** other, (self, ), f"**{other}")

        def _backward():
            self.grad += (other * (self.data ** (other - 1))) * output.grad
        output._backward = _backward
        return output

    def __truediv__(self, other): # self / other
        # a/b = a * (b**-1)
        return self * (other ** -1)

    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self, ), 'tanh')

        def _backward():
            self.grad += ((1 - t**2) * out.grad)
        out._backward = _backward
        return out

    def exp(self):
        x = self.data
        output = Value(math.exp(x), (self, ), 'exp')

        def _backward():
            self.grad += (output.data * output.grad)
        output._backward = _backward
        return output

    def backward(self):
        topo = []
        visited = set()
        def build_topo(v): # v --> root node
            # same as graph traversal logic
            if v not in visited:
                visited.add(v)
                for child in v._prev:
                    build_topo(child)
                topo.append(v)

        build_topo(self)
        self.grad = 1.0
        for node in reversed(topo):
            node._backward()

In [4]:
# visual representation to see the series of operations
from graphviz import Digraph

def trace(root):
    nodes, edges = set(), set() # set of all nodes and edges
    def build(v):
        if v not in nodes: # check each node
            nodes.add(v)
            # connect every child which involved 'v' for that operation 
            # and connect them with an edge
            for child in v._prev: 
                edges.add((child, v))
                build(child)
    build(root)
    return nodes, edges

def draw_dot(root):
    dot = Digraph(format='svg', graph_attr={'rankdir': "LR"}) # left -> right (flow)

    nodes, edges = trace(root)
    for n in nodes:
        uid = str(id(n)) # for unique identification of each node
        # for any value in graph, create rectangular(record) node for it
        dot.node(name = uid, label="{ %s | data %.4f} | grad %.4f" % (n.label, n.data, n.grad), shape='record')
        if n._op:
            # if this value is a result of some operation then create an op node for it
            dot.node(name=uid + n._op, label = n._op)
            # and connect this node to it
            dot.edge(uid + n._op, uid)

    for n1, n2 in edges:
        # connect n1 to the op node of n2
        dot.edge(str(id(n1)), str(id(n2)) + n2._op)

    return dot

In [5]:
import torch
import random

In [6]:
class Neuron:
    def __init__(self, n_inputs):
        # initializing with random values
        self.w = [Value(random.uniform(-1, 1)) for _ in range(n_inputs)]
        self.b = Value(random.uniform(-1, 1))

    def __call__(self, x):
        # wx+b
        activation = sum((wi*xi for wi, xi in zip(self.w, x)), self.b)
        output = activation.tanh()
        return output


class Layer:
    def __init__(self, n_inputs, n_output):
        # fully connected layer
        self.neurons = [Neuron(n_inputs) for _ in range(n_output)]

    def __call__(self, x):
        # perform activation operation for each neuron
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs

In [7]:
class MLP:
    def __init__(self, n_inputs, n_outputs): # n_outs - list of size of layers in MLP
        sz = [n_inputs] + n_outputs
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(n_outputs))]

    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

In [8]:
x = [2.0, 3.0, -1.0]
n = MLP(3, [4, 4, 2])
n(x)

[Value(data=-0.44629211064767604), Value(data=-0.36248586313170916)]

In [9]:
# draw_dot(n(x))

In [10]:
x = [2.0, 3.0]
n = Layer(4, 2)
n(x)

[Value(data=0.9209538164511839), Value(data=-0.9787492251598192)]